In [1]:
from __future__ import print_function
import argparse
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from model import capsules
from loss import SpreadLoss
from datasets import smallNORB


In [2]:
batch_size = 128
test_batch_size = 128
test_intvl = 1
epochs = 10
lr = 3e-3
weight_decay = 2e-7
cuda = True
seed = 1
log_interval = 10
em_iters = 2
snapshot_folder = './snapshots'
data_folder = './data'
dataset = 'mnist'

In [3]:
def get_setting():
    kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
    path = os.path.join(data_folder, dataset)
    if dataset == 'mnist':
        num_class = 10
        train_loader = torch.utils.data.DataLoader(
            datasets.MNIST(path, train=True, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=batch_size, shuffle=True, **kwargs)
        test_loader = torch.utils.data.DataLoader(
            datasets.MNIST(path, train=False,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=test_batch_size, shuffle=True, **kwargs)
    elif dataset == 'smallNORB':
        num_class = 5
        train_loader = torch.utils.data.DataLoader(
            smallNORB(path, train=True, download=True,
                      transform=transforms.Compose([
                          transforms.Resize(48),
                          transforms.RandomCrop(32),
                          transforms.ColorJitter(brightness=32./255, contrast=0.5),
                          transforms.ToTensor()
                      ])),
            batch_size=batch_size, shuffle=True, **kwargs)
        test_loader = torch.utils.data.DataLoader(
            smallNORB(path, train=False,
                      transform=transforms.Compose([
                          transforms.Resize(48),
                          transforms.CenterCrop(32),
                          transforms.ToTensor()
                      ])),
            batch_size=test_batch_size, shuffle=True, **kwargs)
    else:
        raise NameError('Undefined dataset {}'.format(dataset))
    return num_class, train_loader, test_loader


In [4]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


In [5]:
def exp_lr_decay(optimizer, global_step, init_lr = 3e-3, decay_steps = 20000,
                                        decay_rate = 0.96, lr_clip = 3e-3 ,staircase=False):
    
    ''' decayed_learning_rate = learning_rate * decay_rate ^ (global_step / decay_steps)  '''
    
    if staircase:
        lr = (init_lr * decay_rate**(global_step // decay_steps)) 
    else:
        lr = (init_lr * decay_rate**(global_step / decay_steps)) 
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [6]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [7]:
def train(train_loader, model, criterion, optimizer, epoch, device):
# def train(train_loader, model, criterion, optimizer, scheduler, epoch, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()

    model.train()
    train_len = len(train_loader)
    epoch_acc = 0
    end = time.time()

    for batch_idx, (data, target) in enumerate(train_loader):
        data_time.update(time.time() - end)

        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        r = (1.*batch_idx + (epoch-1)*train_len) / (epochs*train_len)
        loss = criterion(output, target, r)
        acc = accuracy(output, target)
        
        global_step = (batch_idx+1) + (epoch - 1) * len(train_loader) 
        exp_lr_decay(optimizer = optimizer, init_lr=lr ,global_step = global_step) # moein - change the learning rate exponentially
        
        
        loss.backward()
        optimizer.step()
        # scheduler.step()

        batch_time.update(time.time() - end)
        end = time.time()

        epoch_acc += acc[0].item()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {}\t[{}/{} ({:.0f}%)]\t'
                  'Loss: {:.6f}\tAccuracy: {:.6f}\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})'.format(
                  epoch, batch_idx * len(data), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader),
                  loss.item(), acc[0].item(),
                  batch_time=batch_time, data_time=data_time))
    return epoch_acc


In [8]:
def snapshot(model, folder, epoch):
    path = os.path.join(folder, 'model_{}.pth'.format(epoch))
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    print('saving model to {}'.format(path))
    torch.save(model.state_dict(), path)


In [9]:
def test(test_loader, model, criterion, device):
    model.eval()
    test_loss = 0
    acc = 0
    test_len = len(test_loader)
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target, r=1).item()
            acc += accuracy(output, target)[0].item()

    test_loss /= test_len
    acc /= test_len
    print('\nTest set: Average loss: {:.6f}, Accuracy: {:.6f} \n'.format(
        test_loss, acc))
    return acc


In [10]:
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

device = torch.device("cuda:0" if cuda else "cpu")

# datasets
num_class, train_loader, test_loader = get_setting()
print(device)

# model
torch.cuda.empty_cache() 
A, B, C, D = 64, 8, 16, 16
# A, B, C, D = 32, 32, 32, 32
model = capsules(A=A, B=B, C=C, D=D, E=num_class,
                 iters=em_iters).to(device)

print(model)

criterion = SpreadLoss(num_class=num_class, m_min=0.2, m_max=0.9)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.96)

best_acc = test(test_loader, model, criterion, device)
torch.cuda.empty_cache()
for epoch in range(1, epochs + 1):
    print('lr: {}'.format(optimizer.param_groups[0]['lr']))
    acc = train(train_loader, model, criterion, optimizer, epoch, device)
#     acc = train(train_loader, model, criterion, optimizer, scheduler, epoch, device)
#     scheduler.step()
    acc /= len(train_loader)
    if epoch % test_intvl == 0:
        best_acc = max(best_acc, test(test_loader, model, criterion, device))
best_acc = max(best_acc, test(test_loader, model, criterion, device))
print('best test accuracy: {:.6f}'.format(best_acc))

snapshot(model, snapshot_folder, epochs)


cuda:0
CapsNet(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (bn1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (primary_caps): PrimaryCaps(
    (pose): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    (a): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
    (sigmoid): Sigmoid()
  )
  (conv_caps1): ConvCaps(
    (sigmoid): Sigmoid()
    (softmax): Softmax(dim=2)
  )
  (conv_caps2): ConvCaps(
    (sigmoid): Sigmoid()
    (softmax): Softmax(dim=2)
  )
  (class_caps): ConvCaps(
    (sigmoid): Sigmoid()
    (softmax): Softmax(dim=2)
  )
)

Test set: Average loss: 7.312526, Accuracy: 11.164953 

lr: 0.003
Train Epoch: 1	[0/60000 (0%)]	Loss: 0.459729	Accuracy: 5.468750	Time 1.657 (1.657)	Data 0.880 (0.880)
Train Epoch: 1	[1280/60000 (2%)]	Loss: 0.252517	Accuracy: 34.375000	Time 0.675 (0.774)	Data 0.000 (0.080)
Train Epoch: 1	[2560/60000 (4%)]	Loss: 0.161386	Accuracy: 67.968750	Time 0.677 (0.731)	D

Train Epoch: 2	[28160/60000 (47%)]	Loss: 0.007306	Accuracy: 100.000000	Time 0.680 (0.693)	Data 0.000 (0.004)
Train Epoch: 2	[29440/60000 (49%)]	Loss: 0.018784	Accuracy: 97.656250	Time 0.681 (0.692)	Data 0.000 (0.004)
Train Epoch: 2	[30720/60000 (51%)]	Loss: 0.028793	Accuracy: 95.312500	Time 0.691 (0.693)	Data 0.000 (0.004)
Train Epoch: 2	[32000/60000 (53%)]	Loss: 0.022048	Accuracy: 97.656250	Time 0.680 (0.692)	Data 0.000 (0.004)
Train Epoch: 2	[33280/60000 (55%)]	Loss: 0.011907	Accuracy: 99.218750	Time 0.684 (0.692)	Data 0.000 (0.004)
Train Epoch: 2	[34560/60000 (58%)]	Loss: 0.018829	Accuracy: 96.093750	Time 0.705 (0.692)	Data 0.000 (0.003)
Train Epoch: 2	[35840/60000 (60%)]	Loss: 0.025814	Accuracy: 96.875000	Time 0.683 (0.693)	Data 0.000 (0.003)
Train Epoch: 2	[37120/60000 (62%)]	Loss: 0.017436	Accuracy: 99.218750	Time 0.681 (0.692)	Data 0.000 (0.003)
Train Epoch: 2	[38400/60000 (64%)]	Loss: 0.022755	Accuracy: 96.875000	Time 0.683 (0.692)	Data 0.001 (0.003)
Train Epoch: 2	[39680/60000

Train Epoch: 4	[3840/60000 (6%)]	Loss: 0.026822	Accuracy: 98.437500	Time 0.678 (0.724)	Data 0.000 (0.028)
Train Epoch: 4	[5120/60000 (9%)]	Loss: 0.015417	Accuracy: 100.000000	Time 0.682 (0.716)	Data 0.001 (0.022)
Train Epoch: 4	[6400/60000 (11%)]	Loss: 0.046325	Accuracy: 97.656250	Time 0.678 (0.711)	Data 0.001 (0.017)
Train Epoch: 4	[7680/60000 (13%)]	Loss: 0.049393	Accuracy: 96.093750	Time 0.679 (0.707)	Data 0.000 (0.015)
Train Epoch: 4	[8960/60000 (15%)]	Loss: 0.038733	Accuracy: 98.437500	Time 0.703 (0.705)	Data 0.000 (0.013)
Train Epoch: 4	[10240/60000 (17%)]	Loss: 0.026222	Accuracy: 100.000000	Time 0.678 (0.703)	Data 0.000 (0.011)
Train Epoch: 4	[11520/60000 (19%)]	Loss: 0.047011	Accuracy: 96.875000	Time 0.679 (0.701)	Data 0.000 (0.010)
Train Epoch: 4	[12800/60000 (21%)]	Loss: 0.046954	Accuracy: 96.093750	Time 0.700 (0.700)	Data 0.000 (0.009)
Train Epoch: 4	[14080/60000 (23%)]	Loss: 0.021823	Accuracy: 100.000000	Time 0.706 (0.700)	Data 0.000 (0.008)
Train Epoch: 4	[15360/60000 (26%

Train Epoch: 5	[40960/60000 (68%)]	Loss: 0.081174	Accuracy: 99.218750	Time 0.681 (0.694)	Data 0.001 (0.003)
Train Epoch: 5	[42240/60000 (70%)]	Loss: 0.095430	Accuracy: 98.437500	Time 0.711 (0.694)	Data 0.000 (0.003)
Train Epoch: 5	[43520/60000 (72%)]	Loss: 0.116041	Accuracy: 99.218750	Time 0.683 (0.694)	Data 0.000 (0.003)
Train Epoch: 5	[44800/60000 (75%)]	Loss: 0.085431	Accuracy: 100.000000	Time 0.678 (0.694)	Data 0.000 (0.003)
Train Epoch: 5	[46080/60000 (77%)]	Loss: 0.095421	Accuracy: 99.218750	Time 0.679 (0.694)	Data 0.000 (0.003)
Train Epoch: 5	[47360/60000 (79%)]	Loss: 0.093506	Accuracy: 100.000000	Time 0.727 (0.694)	Data 0.000 (0.003)
Train Epoch: 5	[48640/60000 (81%)]	Loss: 0.089451	Accuracy: 99.218750	Time 0.695 (0.694)	Data 0.001 (0.003)
Train Epoch: 5	[49920/60000 (83%)]	Loss: 0.119025	Accuracy: 98.437500	Time 0.678 (0.694)	Data 0.000 (0.002)
Train Epoch: 5	[51200/60000 (85%)]	Loss: 0.111504	Accuracy: 98.437500	Time 0.703 (0.694)	Data 0.000 (0.002)
Train Epoch: 5	[52480/6000

Train Epoch: 7	[16640/60000 (28%)]	Loss: 0.264234	Accuracy: 100.000000	Time 0.689 (0.701)	Data 0.000 (0.007)
Train Epoch: 7	[17920/60000 (30%)]	Loss: 0.289550	Accuracy: 99.218750	Time 0.681 (0.699)	Data 0.000 (0.006)
Train Epoch: 7	[19200/60000 (32%)]	Loss: 0.266321	Accuracy: 100.000000	Time 0.681 (0.699)	Data 0.000 (0.006)
Train Epoch: 7	[20480/60000 (34%)]	Loss: 0.293915	Accuracy: 100.000000	Time 0.696 (0.699)	Data 0.000 (0.006)
Train Epoch: 7	[21760/60000 (36%)]	Loss: 0.288465	Accuracy: 100.000000	Time 0.705 (0.698)	Data 0.001 (0.005)
Train Epoch: 7	[23040/60000 (38%)]	Loss: 0.311958	Accuracy: 99.218750	Time 0.697 (0.698)	Data 0.000 (0.005)
Train Epoch: 7	[24320/60000 (41%)]	Loss: 0.343097	Accuracy: 98.437500	Time 0.683 (0.698)	Data 0.001 (0.005)
Train Epoch: 7	[25600/60000 (43%)]	Loss: 0.298935	Accuracy: 99.218750	Time 0.709 (0.698)	Data 0.000 (0.005)
Train Epoch: 7	[26880/60000 (45%)]	Loss: 0.305386	Accuracy: 99.218750	Time 0.678 (0.697)	Data 0.000 (0.004)
Train Epoch: 7	[28160/60

Train Epoch: 8	[53760/60000 (90%)]	Loss: 0.719214	Accuracy: 97.656250	Time 0.680 (0.694)	Data 0.000 (0.002)
Train Epoch: 8	[55040/60000 (92%)]	Loss: 0.681956	Accuracy: 99.218750	Time 0.679 (0.694)	Data 0.000 (0.002)
Train Epoch: 8	[56320/60000 (94%)]	Loss: 0.681366	Accuracy: 98.437500	Time 0.744 (0.694)	Data 0.000 (0.002)
Train Epoch: 8	[57600/60000 (96%)]	Loss: 0.724193	Accuracy: 97.656250	Time 0.690 (0.694)	Data 0.000 (0.002)
Train Epoch: 8	[58880/60000 (98%)]	Loss: 0.700563	Accuracy: 99.218750	Time 0.786 (0.694)	Data 0.001 (0.002)

Test set: Average loss: 1.605779, Accuracy: 98.457278 

lr: 0.002977113129526248
Train Epoch: 9	[0/60000 (0%)]	Loss: 0.702593	Accuracy: 99.218750	Time 1.568 (1.568)	Data 0.882 (0.882)
Train Epoch: 9	[1280/60000 (2%)]	Loss: 0.688876	Accuracy: 100.000000	Time 0.679 (0.780)	Data 0.000 (0.080)
Train Epoch: 9	[2560/60000 (4%)]	Loss: 0.673234	Accuracy: 99.218750	Time 0.689 (0.741)	Data 0.001 (0.042)
Train Epoch: 9	[3840/60000 (6%)]	Loss: 0.676055	Accuracy: 99.2

Train Epoch: 10	[29440/60000 (49%)]	Loss: 1.187950	Accuracy: 99.218750	Time 0.679 (0.697)	Data 0.000 (0.004)
Train Epoch: 10	[30720/60000 (51%)]	Loss: 1.187324	Accuracy: 98.437500	Time 0.679 (0.696)	Data 0.000 (0.004)
Train Epoch: 10	[32000/60000 (53%)]	Loss: 1.201239	Accuracy: 99.218750	Time 0.677 (0.696)	Data 0.000 (0.004)
Train Epoch: 10	[33280/60000 (55%)]	Loss: 1.178184	Accuracy: 99.218750	Time 0.785 (0.696)	Data 0.000 (0.004)
Train Epoch: 10	[34560/60000 (58%)]	Loss: 1.204934	Accuracy: 100.000000	Time 0.679 (0.696)	Data 0.000 (0.004)
Train Epoch: 10	[35840/60000 (60%)]	Loss: 1.259270	Accuracy: 99.218750	Time 0.704 (0.697)	Data 0.000 (0.003)
Train Epoch: 10	[37120/60000 (62%)]	Loss: 1.214055	Accuracy: 99.218750	Time 0.723 (0.697)	Data 0.000 (0.003)
Train Epoch: 10	[38400/60000 (64%)]	Loss: 1.233716	Accuracy: 99.218750	Time 0.689 (0.696)	Data 0.000 (0.003)
Train Epoch: 10	[39680/60000 (66%)]	Loss: 1.247237	Accuracy: 98.437500	Time 0.722 (0.697)	Data 0.001 (0.003)
Train Epoch: 10	[4

In [11]:
# !python train.py --lr=0.01 --weight-decay=0. --epochs=30 --batch-size=16
# %run -i train --lr=0.01 --weight_decay=0. --epochs=30 --batch-size=64